# Precision Medicine Workshop

<a href="https://githubtocolab.com/ARCLeeds/pm_workshop/blob/main/Precision_Medicine_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

## Make sure you run the cell below first (`%load_ext rpy2.ipython`). This makes this Colab notebook able to run R in the cells. Run the code in each grey cell by clicking the arrow to the left that appears in the square brackets. You can make changes to the code if you want.

In [ ]:
%load_ext rpy2.ipython

## Now download the data files we need into this notebook.



In [ ]:
!git clone https://github.com/ARCLeeds/pm_workshop.git

In [ ]:
%%R
#Data to read in:
imiss<-read.table(file="pm_workshop/testplink.imiss",header=TRUE)
lmiss<-read.table(file="pm_workshop/testplink.lmiss",header=TRUE)
hetph<-read.table(file="pm_workshop/testplink_ph.het",header=TRUE,sep="\t")
pcaph<-read.table(file="pm_workshop/testplink_ph.eigenvec",header=TRUE,sep="\t")
pca2ph<-read.table(file="pm_workshop/testplinkasian_ph.eigenvec",header=TRUE,sep="\t")
reslog<-read.table(file="pm_workshop/testplinkallWITHCUMPOS.assoc.logistic",header=TRUE)
reslog2<-read.table(file="pm_workshop/testplinkall2WITHCUMPOS.assoc.logistic",header=TRUE)

## These are data from the Born in Bradford project. It includes data on almost 1,000 people at >84,000 genetic variants from across the genome. It includes individuals from multiple ethnicities, most commonly self-declaring themselves 'white British' or 'Pakistani'.

## We'll look at some *quality control* metrics to ensure the data are of good quality, then see what genetic data can tell us about *ethnicity*. Finally we'll conduct *tests of association* between every genetic marker and a disease (melanoma).

## **Quality control - missingness**

## First we look at how much *missingness* there is in the data  - per marker and per person (an indicator of quality) - by plotting histograms of these.

In [ ]:
%%R
imiss<-read.table(file="pm_workshop/testplink.imiss",header=TRUE)
lmiss<-read.table(file="pm_workshop/testplink.lmiss",header=TRUE)

par(mfrow=c(2,1))
hist(imiss[,6],xlab="Missingness (%)",breaks=seq(from=0,to=0.044,by=0.002),main="Missingness per marker",col="green")
hist(lmiss[,5],xlab="Missingness (%)",breaks=seq(from=0,to=0.044,by=0.002),main="Missingness per person",col="red")



## Try changing the colours of the histograms and rerunning

## Let's see how many people are missing genotypes at > 3% of their genetic variants


In [ ]:
%%R
nrow(imiss[imiss[,6]>0.03,])


## Let's see how many genetic variants are missing in > 3% of people

In [ ]:
%%R
nrow(lmiss[lmiss[,5]>0.03,])

## Try changing the missingness thresholds from 3% to something different and rerunning

## **Quality control - homozygosity**

## Everyone carries two copies (alleles) of each genetic variant (one from their mother and one from their father).
## A standard measure of genotype quality is to look at homozygosity - what proportion of the genome is the same on both (maternal and paternal) chromosomes.
## If genotyping isn't working well then often only one chromosome produces a signal and it looks like this person has a lot of homozygosity.

In [ ]:
%%R

hetph<-read.table(file="pm_workshop/testplink_ph.het",header=TRUE,sep="\t")



## Let's plot a histogram of homozygosity

In [ ]:
%%R
par(mfrow=c(1,1))
hist(hetph[,6],main="Homozygosity",xlab="Homozygosity",col="blue",xlim=c(-0.3,0.3))

This is a bigger spread to the right than we would normally expect - i.e. excess homozygosity. Let's try looking separately in the two most common ethnicities

In [ ]:
%%R
hist(hetph[!is.na(hetph[,7])&hetph[,7]=="White - British",6],main="Homozygosity - White British",breaks=seq(from=-0.3,to=0.3,by=0.025),xlab="Homozygosity",col="blue",xlim=c(-0.3,0.3))


In [ ]:
%%R
hist(hetph[!is.na(hetph[,7])&hetph[,7]=="Asian - Pakistani",6],main="Homozygosity - Pakistani",breaks=seq(from=-0.3,to=0.3,by=0.025),xlab="Homozygosity",col="red",xlim=c(-0.3,0.3))

## Clearly the excess homozygosity is in the Pakistani group. Why do you think this might be?

## **Ethnicity**

## Self-declared ethnicity is not always reliable, especially when we are looking for subtle differences (e.g. different coutnries in Europe).
## We can look for clusters of individuals based on genetic data.
## In a mixed ethnicity sample, we expect that those variants that are most variable are the ones that differ in frequency by ethnicity.
## Principal components analysis (PCA) will calculate a weighted average across an individual's genotypes, giving the biggest weights to the most variable genetic variants.
## PC1 is the weighted average of genotypes that captures the maximum amount of variation in the dataset.
## PC2  is the weighted average of genotypes that captures the next greatest amount of variation.
## If we plot PC2 against PC1 we should see those individuals with similar ethnicity cluster together

In [ ]:
%%R

pcaph<-read.table(file="pm_workshop/testplink_ph.eigenvec",header=TRUE,sep="\t")

par(mfrow=c(1,1))
plot(pcaph[,2],pcaph[,3],xlab="PC1",ylab="PC2")

#points(pcaph[pcaph[,4]=="Asian - Bangladeshi",2],pcaph[pcaph[,4]=="Asian - Bangladeshi",3],col="red")
#points(pcaph[pcaph[,4]=="Asian - Indian",2],pcaph[pcaph[,4]=="Asian - Indian",3],col="red")
#points(pcaph[pcaph[,4]=="Asian - Pakistani",2],pcaph[pcaph[,4]=="Asian - Pakistani",3],col="red")
#points(pcaph[pcaph[,4]=="Black - African",2],pcaph[pcaph[,4]=="Black - African",3],col="magenta")
#points(pcaph[pcaph[,4]=="Other",2],pcaph[pcaph[,4]=="Other",3],col="red")
#points(pcaph[pcaph[,4]=="White - British",2],pcaph[pcaph[,4]=="White - British",3],col="green")
#points(pcaph[pcaph[,4]=="White - Other",2],pcaph[pcaph[,4]=="White - Other",3],col="red")

#Possible colours: red, green, blue, yellow, magenta, purple, grey, skyblue, orange


## Each point represents an individual.
## Individuals who are close together are from similar ethnic groups.
## Remove the hash symbols to plot individuals from different ethnic groups in different colours to see where they lie on the plot.
## Try plotting them in different colours so you can distinguish them.

## We can run the same analysis just for the south Asian samples.
## Although they are all from the same region of Pakistan, marriages tend to be within biraderi (clans).
## Let's see whether there is any genetic clustering based on these biraderi.

In [ ]:
%%R

pca2ph<-read.table(file="pm_workshop/testplinkasian_ph.eigenvec",header=TRUE,sep="\t")

plot(pca2ph[,2],pca2ph[,3],xlab="PC1",ylab="PC2")

#points(pca2ph[pca2ph[,5]=="Bains",2],pca2ph[pca2ph[,5]=="Bains",3],col="red")
#points(pca2ph[pca2ph[,5]=="Choudhry",2],pca2ph[pca2ph[,5]=="Choudhry",3],col="red")
#points(pca2ph[pca2ph[,5]=="Jatt",2],pca2ph[pca2ph[,5]=="Jatt",3],col="red")
#points(pca2ph[pca2ph[,5]=="Kashmiri",2],pca2ph[pca2ph[,5]=="Kashmiri",3],col="red")
#points(pca2ph[pca2ph[,5]=="Mughal",2],pca2ph[pca2ph[,5]=="Mughal",3],col="red")
#points(pca2ph[pca2ph[,5]=="Pathan",2],pca2ph[pca2ph[,5]=="Pathan",3],col="red")
#points(pca2ph[pca2ph[,5]=="Rajput",2],pca2ph[pca2ph[,5]=="Rajput",3],col="red")

## There appear to be clusters here.
## Once again remove the hash symbols (and try different colours) to see where the different biarderis are on this plot.
## Which biraderis look most different and which most similar (see the bottom of the page for more reading about this)?

## **Testing for association between genetic variants and disease risk**

## Testing for genetic association with disease is conceptually easy.
## If a variant increases risk of disease then it will be more frequent among cases.
## So we just collect cases and controls for a disease and test every genetic variant to see whether they differ in frequency between these two groups.
## The main reason we are concerned about ethnicity when thinking about disease risk is *confounding*.
## Imagine most of our cases come from one ethnicity and most of our controls from a different ethnicity.
## Any genetic marker associated with disease risk will differ between the two groups, but so will any genetic marker that differs in frequency between the two groups.

## Here we read in and plot the results from a test between melanoma cases and controls in our dataset.

In [ ]:
%%R

reslog<-read.table(file="pm_workshop/testplinkallWITHCUMPOS.assoc.logistic",header=TRUE)

plot(reslog[round(0.5*reslog[,1])==0.5*reslog[,1],10],-log10(reslog[round(0.5*reslog[,1])==0.5*reslog[,1],9]),col="cornflowerblue",xlim=c(min(reslog[,10],na.rm=T),max(reslog[,10],na.rm=T)),xaxt="n",xlab="position",ylab="-log10p")
points(reslog[round(0.5*reslog[,1])!=0.5*reslog[,1],10],-log10(reslog[round(0.5*reslog[,1])!=0.5*reslog[,1],9]),col="darkblue")

abline(h=-log10(5*(10^(-8))),col="red")

## Every point represents a genetic marker.
## They are plotted in order of their position across the genome on the X-axis.
## On the Y-axis is plotted the -log of the p-value - a value of 2 is a p-value of 0.01, a value of 3 is a p-value of 0.001, etc.
## In genetic studies we say that a marker is associated if it's p-value < 0.00000005 (-logp > 7.3). So every point above the red line is significantly associated with disease.
## What do you notice?

## **Confounding!**

##Let's look at how case frequency differs by ethnicity by tabulating this
## Cases are indicated by a '2' and controls by a '1'


In [ ]:
%%R

table(pcaph[,c(4,7)])

##All the cases are in 'white British' samples (it's melanoma!).
## It's likely that most of the genetic variants that look 'significant' are those that are more frequent in white British samples compared to south Asian.
##So let try looking at the results when we just have white British samples:

In [ ]:
%%R

reslog2<-read.table(file="pm_workshop/testplinkall2WITHCUMPOS.assoc.logistic",header=TRUE)

plot(reslog2[round(0.5*reslog2[,1])==0.5*reslog2[,1],10],-log10(reslog2[round(0.5*reslog2[,1])==0.5*reslog2[,1],9]),col="cornflowerblue",xlim=c(min(reslog2[,10],na.rm=T),max(reslog2[,10],na.rm=T)),xaxt="n",xlab="position",ylab="-log10p")
points(reslog2[round(0.5*reslog2[,1])!=0.5*reslog2[,1],10],-log10(reslog2[round(0.5*reslog2[,1])!=0.5*reslog2[,1],9]),col="darkblue")

abline(h=-log10(5*(10^(-8))),col="red")

##We can pick out the variants that are significantly associated with melanoma risk


In [ ]:
%%R

reslog2[!is.na(reslog2[,9])&reslog2[,9]<0.000001,]

## The two chromosome 9 variants are in *CDKN2A*, a tumour suppressor gene. Mutations in *CDKN2A* are found in ~20% of familial melanoma cases.
## The chromosome 16 variant is close to *MC1R* - the gene that most strongly influences the red hair/pale skin/freckly phenotype.
## No surprise to find these associated with melanoma!

## **More about the ethnicity results**

**Pathan** are quite distinct and historically come from a different part of Pakistan.

**Jatt** and **Choudhry** seem to overlap completely.

Choudhry is an honorary title in Punjab and Kashmir used most commonly by the Jatts.

**Bains** is somewhat separate to others, while Rajput overlaps both **Bains** and **Jatt/Choudhry**.

It could be that **Rajput** has different subgroups or that it is a title taken on by people with different ancestry.

For more about this see my article in Nature Communications: 

[Arcerio et al (2021) Fine-scale population structure and demographic history of British Pakistanis. Nat Commun 12(1):7189](https://www.nature.com/articles/s41467-021-27394-2)

## **More about the melanoma results**

For more about melanoma genetics see my article in Nature Genetics:

[Landi et al (2020) Genome-wide association meta-analyses combining multiple risk phenotypes provide insights into the genetic architecture of cutaneous melanoma susceptibility. 52(5):494-504](https://www.nature.com/articles/s41588-020-0611-8)